In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser
import pandas as pd
import time

In [2]:
url = 'http://ncov.mohw.go.kr/tcmBoardList.do?pageIndex=1&brdId=&brdGubun=&board_id=&search_item=1&search_content=%EA%B5%AD%EB%82%B4+%EB%B0%9C%EC%83%9D+%ED%98%84%ED%99%A9+'
j=3
n=0
df0 = pd.DataFrame()
df1 = pd.DataFrame()
    
browser = webdriver.Chrome('E:/tj/jupyter_notebook/chromedriver.exe')
browser.get(url)
try:
    while n<2:

        for i in range(1,11):
            title = browser.find_element_by_xpath('//*[@id="content"]/div/div[4]/table/tbody/tr[%s]/td[2]/a'%i)
            text = title.get_attribute('text')
            if '정례브리핑' in text:
                browser.find_element_by_xpath('//*[@id="content"]/div/div[4]/table/tbody/tr[%s]/td[2]/a'%i).click()
                time.sleep(1)
                #데이터프레임 생성
                result_html = browser.page_source
                result_soup = BeautifulSoup(result_html, 'html.parser')
                tags = result_soup.find_all("table")
                Time = result_soup.find_all("span", attrs={"class":"bvc_detail"})[1]
                
                if str(Time.get_text())[:10] == '2020-02-14':
                    break
                
                html_table = parser.make2d(tags[0])
                df0_1=pd.DataFrame(html_table[2:3], columns=html_table[1])
                df0_1['time'] = str(Time.get_text())[:10]
                if len(df0_1.columns)>9:
                    df0_1 = df0_1.drop('계',axis=1)
                df0_1.columns = ['구분','총계','확진자','격리해제','격리중','사망','검사중','결과음성','time']
                df0 = pd.concat([df0,df0_1],axis=0)


                html_table = parser.make2d(tags[1])
                df1_1=pd.DataFrame(html_table[1:-1], columns=html_table[0])
                if len(df1_1.columns)==19:
                    df1_1['검역']=0
                df1_1.columns = ['구분','합계','서울','부산','대구','인천','광주','대전','울산','세종','경기','강원','충북','충남','전북','전남','경북','경남','제주','검역']
                df1_1['time'] = str(Time.get_text())[:10]
                df1 = pd.concat([df1,df1_1],axis=0)

                browser.back()
        browser.find_element_by_xpath('//*[@id="content"]/div/div[5]/a[%s]'%j).click()
        time.sleep(1)
        j +=1
        if j == 13:
            j=3
            n+=1
        #포문 종료
except:
    browser.close()

In [5]:
df0.to_csv('확진자.csv')

In [6]:
df1.to_csv('지역별.csv')